## Swizzle some tensors

Illustrate swizzling the ranks of a tensor by name.

First, include some libraries


In [ ]:
%run ../prelude.py --style=tree --animation=none

## Initialize setup

The following cell just creates some sliders to control the creation of the input operand tensors.

In [ ]:
# Initial values

M = 8
K = 6
density = [0.9, 0.8]
seed = 10

def set_params(rank_M, rank_K, tensor_density, rand_seed):
    global M
    global K
    global density
    global seed
    
    M = rank_M
    K = rank_K
    
    if tensor_density == 'sparse':
        density = [0.9, 0.8]
    elif tensor_density == 'sparser':
        density = [0.9, 0.4]
    else:
        density = [1.0, 1.0]
        
    seed = rand_seed

interactive(set_params,
            rank_M=widgets.IntSlider(min=2, max=12, step=1, value=M),
            rank_K=widgets.IntSlider(min=2, max=12, step=1, value=K),
            tensor_density=['sparser', 'sparse', 'dense'],
            rand_seed=widgets.IntSlider(min=0, max=100, step=1, value=seed))

## Create Input Tensors

Because this notebook tends to use both the original and rank swapped versions of the operands, the tensor names are suffixed with the ordered names of the ranks.


In [ ]:
a_MK = Tensor.fromRandom(["M", "K"], [M, K], density, 5, seed=seed)
a_MK.setColor("blue").setName("a_MK")
displayTensor(a_MK)




## Create some reference tensors

In [ ]:
a_MK = Tensor.fromUncompressed(["M", "K"], 
                               [[0, 0, 4, 0, 0, 5],
                                [3, 2, 0, 3, 0, 2],
                                [0, 2, 0, 0, 1, 2],
                                [0, 0, 0, 0, 0, 0],
                                [2, 5, 0, 0, 0, 5],
                                [4, 1, 0, 0, 0, 0],
                                [5, 0, 0, 1, 0, 0],
                                [4, 0, 0, 5, 1, 3]])

# Create swapped rank version of a
a_KM = a_MK.swapRanks()
a_KM.setName("a_KM")
displayTensor(a_KM)

M1 = 2
M0 = (M+1)//M1

K1 = 2
K0 = (K+1)//K1

a_MMKK = a_MK.splitUniform(M0).splitUniform(K0, depth=2)
a_MMKK.setName("a_MMKK")
displayTensor(a_MMKK)

a_MKMK = a_MMKK.swapRanks(depth=1)
a_MKMK.setName("a_MKMK")
displayTensor(a_MKMK)

a_KMMK = a_KM.splitUniform(K0).swapRanks(depth=1).splitUniform(M0, depth=1)
a_KMMK.setName("a_KMMK")
displayTensor(a_KMMK)

In [ ]:
a_KM_2 = a_MK.swizzleRanks(["K", "M"])
displayTensor(a_KM_2)
print(a_KM_2 == a_KM)

a_MK_2 = a_KM_2.swizzleRanks(["M", "K"])
displayTensor(a_MK_2)
print(a_MK_2 == a_MK)

a_MKMK_2 = a_MMKK.swizzleRanks(["M.1","K.1", "M.0", "K.0"])
displayTensor(a_MKMK_2)
print(a_MKMK_2 == a_MKMK)

a_MMKK_2 = a_MKMK.swizzleRanks(["M.1", "M.0", "K.1", "K.0"])
displayTensor(a_MMKK_2)
print(a_MMKK_2 == a_MMKK)

a_M0M1MK0K1 = a_MMKK.swizzleRanks(["M.0", "M.1", "K.0", "K.1"])
displayTensor(a_M0M1MK0K1)

## Testing Area